# * Interim : 2024 Target Sales DTAC (Re-org 65 -> 64 Province)
    Jan - May 2024 (K.Voraphon)
    begin Jun 2024 (K.Wanicha)

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 3, 24)

## ETL Process...

### Step 1 : Import Data Source

In [3]:
''' Rawdata '''

# Target_Sales_DTAC_Y2024
src_file = '../../data/interim/Target_Sales_DTAC_Y2024.xlsx'
src_sheet = 'Rawdata'
src_col = ['DATA_DATE', 'DATA_MONTH', 'DATA_YEAR', 'METRIC_VALUES', 'COMP_CD', 'VERSION', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_DESC', 'CHANNEL_CD']
src_df = pd.read_excel(src_file, sheet_name=src_sheet, usecols=src_col, index_col=None) 
src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'DATA_MONTH': 'MONTH_SHORT', 'DATA_YEAR': 'TM_KEY_YR', 'METRIC_VALUES': 'MTH_VALUE'}, inplace=True)
src_df['MONTH_SHORT'] = src_df['MONTH_SHORT'].apply(lambda x: x.upper())
# src_df = src_df.replace(np.nan, None)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df.tail(3)


src_df : 55692 rows, 12 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
55689,20241218,DEC,2024,DB1R000900AK,Prepaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,T,HH,906111,TRANG,AK
55690,20241218,DEC,2024,DB1R000900AK,Prepaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,T,HH,906112,YALA,AK
55691,20241218,DEC,2024,DB1R000900AK,Prepaid Inflow M1 : DTAC : Wholesales,50502655.0,DTAC,T,True corp,True corp,True corp,AK


In [5]:
''' Master Data '''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)
dt_df['MONTH_SHORT'] = dt_df['MONTH_SHORT'].apply(lambda x: x.upper())
# dt_df['MONTH_KEY'] = dt_df['MONTH_SHORT'].str.upper()
# dt_df.tail(3)

# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
mooc_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['HOP_HINT']!='True Corp')]
# mooc_df.tail(3)

# Create H level
mooc_h_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT']].drop_duplicates()
mooc_h_df.dropna(how='all', inplace=True)
mooc_h_df['AREA_KEY'] = mooc_h_df['ORGID_H'].apply(lambda x: x if re.search(r'[a-z]', x, re.I) else int(x))
# mooc_h_df.tail(3)

# Create HH level
mooc_hh_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER']].drop_duplicates()
mooc_hh_df.dropna(how='all', inplace=True)
mooc_hh_df['AREA_KEY'] = mooc_hh_df['ORGID_HH'].astype(int).astype(str)
# mooc_hh_df.tail(3)

In [6]:
''' Example DataFrame '''

src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)
# mooc_h_df.loc[mooc_h_df['ORGID_H'].str.contains('^0')].tail(3)

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'MTH_VALUE': 'sum', 'AREA_TYPE': 'nunique', 'AREA_KEY': 'nunique'}).reset_index()
# tmp_df

,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
55689,20241218,DEC,2024,DB1R000900AK,Prepaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,T,HH,906111,TRANG,AK
55690,20241218,DEC,2024,DB1R000900AK,Prepaid Inflow M1 : DTAC : Wholesales,0.0,DTAC,T,HH,906112,YALA,AK
55691,20241218,DEC,2024,DB1R000900AK,Prepaid Inflow M1 : DTAC : Wholesales,50502655.0,DTAC,T,True corp,True corp,True corp,AK


### Step 2 : Aggregate Data

In [11]:
''' ALL Channel '''


def all_chn_metric(v_cd, v_name):

    cd = v_cd
    name = v_name
    two_result = []
    if cd == 'DB1R000900': two_result = 'DB1R000900CS', 'Prepaid Inflow M1 : DTAC'
    elif cd == 'DB1S000101': two_result = 'DB1S000101CS', 'Prepaid Gross Adds : DTAC'
    elif cd == 'DB2R000500': two_result = 'DB2R010500CS', 'Postpaid Inflow M1 B2C : DTAC'
    elif cd == 'DB2S000100': two_result = 'DB2S010100CS', 'Postpaid Gross Adds B2C : DTAC'
    else: two_result = cd, name
    return two_result


''' Filter '''
all_chn_df = src_df.copy()
all_chn_df = all_chn_df.loc[all_chn_df['METRIC_CD'].isin(['DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100'])]

''' Modify '''
all_chn_df[['METRIC_CD', 'METRIC_NAME']] = all_chn_df.apply(lambda x: pd.Series(all_chn_metric(x['METRIC_CD'], x['METRIC_NAME'])), axis=1)

''' Sort '''
all_chn_df = all_chn_df.reset_index(drop=True)

# ''' Test : Sum ALL & Sort '''
# test_all_chn_df = all_chn_df.groupby(['DATA_DATE', 'MONTH_SHORT', 'TM_KEY_YR', 'COMP_CD', 'VERSION', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'CHANNEL_CD'])\
#     .agg({'MTH_VALUE': 'sum'}).sort_values(by=['MONTH_SHORT', 'METRIC_CD', 'AREA_TYPE']).reset_index()
# test_all_chn_df

print(f'\nall_chn_df : {all_chn_df.shape[0]} rows, {all_chn_df.shape[1]} columns')
all_chn_df.tail(3)


all_chn_df : 4284 rows, 12 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
4281,20241218,DEC,2024,DB1R000900CS,Prepaid Inflow M1 : DTAC,1.170782e+06,DTAC,T,HH,906111,TRANG,ALL
4282,20241218,DEC,2024,DB1R000900CS,Prepaid Inflow M1 : DTAC,9.880384e+05,DTAC,T,HH,906112,YALA,ALL
4283,20241218,DEC,2024,DB1R000900CS,Prepaid Inflow M1 : DTAC,5.784005e+07,DTAC,T,True corp,True corp,True corp,ALL


In [12]:
''' GEO Channel

    AA  : Account Executive
    AC  : Branded Retail
    AE  : Direct Sales
    AGA : Modern Trade Synergy
    AJ  : Retail Sales
    AK  : Wholesales
'''


def geo_chn_metric(v_cd, v_name):

    cd = v_cd
    name = v_name
    two_result = []
    if re.search('DB1R000900', cd): two_result = 'DB1R000900CG', 'Prepaid Inflow M1 : DTAC - GEO Channel'
    elif re.search('DB1S000101', cd): two_result = 'DB1S000101CG', 'Prepaid Gross Adds : DTAC - GEO Channel'
    elif re.search('DB2R000500', cd): two_result = 'DB2R010500CG', 'Postpaid Inflow M1 B2C : DTAC - GEO Channel'
    elif re.search('DB2S000100', cd): two_result = 'DB2S010100CG', 'Postpaid Gross Adds B2C : DTAC - GEO Channel'
    else: two_result = cd, name
    return two_result


''' Filter '''
geo_chn_df = src_df.copy()
# geo_chn_df = geo_chn_df.loc[geo_chn_df['METRIC_CD'].str.contains('^DB1R000900|^DB1S000101|^DB2R000500|^DB2S000100') & geo_chn_df['METRIC_CD'].str.contains('AA$|AC$|AE$|AGA$|AJ$|AK$')]
geo_chn_df = geo_chn_df.loc[geo_chn_df['METRIC_CD'].str.contains('^DB1R000900|^DB1S000101|^DB2R000500|^DB2S000100') & geo_chn_df['METRIC_CD'].str.contains('AC$|AE$|AGA$|AJ$')]

''' Modify '''
geo_chn_df[['METRIC_CD', 'METRIC_NAME']] = geo_chn_df.apply(lambda x: pd.Series(geo_chn_metric(x['METRIC_CD'], x['METRIC_NAME'])), axis=1)
geo_chn_df['CHANNEL_CD'] = 'GEO'

''' Data Test '''
# geo_chn_df = geo_chn_df.loc[geo_chn_df['MONTH_SHORT']=='JAN']
# geo_chn_df = geo_chn_df.loc[geo_chn_df['METRIC_CD']=='DB1S000101CG']
# geo_chn_df = geo_chn_df.loc[geo_chn_df['AREA_KEY']=='902033']

''' Sum GEO & Sort '''
geo_chn_df = geo_chn_df.groupby(['DATA_DATE', 'MONTH_SHORT', 'TM_KEY_YR', 'COMP_CD', 'VERSION', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'CHANNEL_CD']).agg({'MTH_VALUE': 'sum'}).reset_index()

print(f'\ngeo_chn_df : {geo_chn_df.shape[0]} rows, {geo_chn_df.shape[1]} columns')
geo_chn_df.tail(3)


geo_chn_df : 4272 rows, 12 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,COMP_CD,VERSION,METRIC_CD,METRIC_NAME,AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD,MTH_VALUE
4269,20241218,DEC,2024,DTAC,T,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,HH,910097,UBON RATCHATHANI,GEO,630.173005
4270,20241218,DEC,2024,DTAC,T,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,HH,910098,YASOTHON,GEO,0.000000
4271,20241218,DEC,2024,DTAC,T,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,True corp,True corp,True corp,GEO,0.000000


In [14]:
''' Concat ALL & GEO'''

raw_df = pd.concat([all_chn_df, geo_chn_df])
raw_df

,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
0,20240523,JAN,2024,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,1208.000000,DTAC,T,H,50,"BKK : Bang Khun Thian, Chom Thong, Bang Bon",ALL
1,20240523,JAN,2024,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,613.000000,DTAC,T,H,80,"BKK : Thawi Watthana, Taling Chan, Phasi Charo...",ALL
2,20240523,JAN,2024,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,720.000000,DTAC,T,H,80,"BKK : Thawi Watthana, Taling Chan, Phasi Charo...",ALL
3,20240523,JAN,2024,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,277.000000,DTAC,T,H,49,"BKK : Thon Buri, Khlong San, Rat Burana, Thung...",ALL
4,20240523,JAN,2024,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,944.000000,DTAC,T,H,202,"NTB : Bang Kruai, Bang Yai",ALL
...,...,...,...,...,...,...,...,...,...,...,...,...
4267,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,178.549303,DTAC,T,HH,910095,SI SA KET,GEO
4268,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,121.547647,DTAC,T,HH,910096,SURIN,GEO
4269,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,630.173005,DTAC,T,HH,910097,UBON RATCHATHANI,GEO
4270,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,0.000000,DTAC,T,HH,910098,YASOTHON,GEO


In [25]:
''' Check Re-Org (016, 040) to 080 '''

test_raw_df = raw_df.groupby(['DATA_DATE', 'MONTH_SHORT', 'TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC']).agg({'MTH_VALUE': 'sum'}).reset_index()
# test_raw_df = test_raw_df.loc[test_raw_df['AREA_TYPE']=='True corp']
test_raw_df = test_raw_df.loc[test_raw_df['AREA_KEY']==80]
test_raw_df.tail(3)

,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,MTH_VALUE
1382,20240523,MAR,2024,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,DTAC,T,H,80,"BKK : Thawi Watthana, Taling Chan, Phasi Charo...",801400.0
1447,20240523,MAR,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,DTAC,T,H,80,"BKK : Thawi Watthana, Taling Chan, Phasi Charo...",837.0
1512,20240523,MAR,2024,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,DTAC,T,H,80,"BKK : Thawi Watthana, Taling Chan, Phasi Charo...",1301.0


In [26]:
''' Join Area '''

corp_df = raw_df.loc[raw_df['AREA_TYPE']=='True corp']
# corp_df.tail(3)

merge_h_df = pd.merge(raw_df.loc[raw_df['AREA_TYPE']=='H'], mooc_h_df, how='left', on='AREA_KEY')
# merge_h_df = merge_h_df.loc[merge_h_df['AREA_KEY']==80]
# merge_h_df.tail(3)

merge_hh_df = pd.merge(raw_df.loc[raw_df['AREA_TYPE']=='HH'], mooc_hh_df, how='left', on='AREA_KEY')
# merge_hh_df.tail(3)

merge_area_df = pd.concat([corp_df, merge_h_df, merge_hh_df])
print(f'\nprep_agg_df : {merge_area_df.shape[0]} rows, {merge_area_df.shape[1]} columns')
merge_area_df.tail(3)


prep_agg_df : 8556 rows, 19 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER
6909,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,121.547647,DTAC,T,HH,910096,SURIN,GEO,UPC,GX6,Regional Management 6 (Northeast 2),33X,"SURIN, SI SA KET",910096.0,SURIN
6910,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,630.173005,DTAC,T,HH,910097,UBON RATCHATHANI,GEO,UPC,GX6,Regional Management 6 (Northeast 2),34Z,UBON RATCHATHANI,910097.0,UBON RATCHATHANI
6911,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,0.000000,DTAC,T,HH,910098,YASOTHON,GEO,UPC,GX6,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON


In [27]:
''' Join Period '''

merge_period_df = pd.merge(merge_area_df, dt_df, how='left', on=['TM_KEY_YR', 'MONTH_SHORT'])
# merge_period_df.tail(3)

# Prep Aggregate
prep_agg_df = merge_period_df
prep_agg_df['DAY_VALUE'] = prep_agg_df['MTH_VALUE'] / prep_agg_df['DAYS_IN_MONTH']
print(f'\nprep_agg_df : {prep_agg_df.shape[0]} rows, {prep_agg_df.shape[1]} columns')
prep_agg_df.tail(3)


prep_agg_df : 261084 rows, 24 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,...,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,TM_KEY_DAY,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TM_KEY_MTH,DAY_VALUE
261081,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,0.0,DTAC,T,HH,910098,...,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON,20241229,31,2024052,202412,0.0
261082,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,0.0,DTAC,T,HH,910098,...,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON,20241230,31,2025001,202412,0.0
261083,20241218,DEC,2024,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,0.0,DTAC,T,HH,910098,...,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON,20241231,31,2025001,202412,0.0


In [28]:
''' Aggregate P, G, H, HH '''

agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'

# P : Nationwide
agg_p_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_p_df['AREA_NO'] = 1
agg_p_df['AREA_TYPE'] = 'P'
agg_p_df['AREA_CD'] = 'P'
agg_p_df['AREA_NAME'] = 'Nationwide'
agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

# G : Region
agg_g_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_G', 'TDS_SGMD']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_g_df['AREA_NO'] = 2
agg_g_df['AREA_TYPE'] = 'G'
agg_g_df.rename(columns={'ORGID_G': 'AREA_CD'}, inplace=True)
agg_g_df.rename(columns={'TDS_SGMD': 'AREA_NAME'}, inplace=True)
agg_g_df = agg_g_df.loc[:, agg_cols]
# agg_g_df[agg_g_df['TM_KEY_DAY']==20240501]

# H : HOP_HINT
agg_h_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_H', 'HOP_HINT']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_h_df['AREA_NO'] = 3
agg_h_df['AREA_TYPE'] = 'H'
agg_h_df.rename(columns={'ORGID_H': 'AREA_CD'}, inplace=True)
agg_h_df.rename(columns={'HOP_HINT': 'AREA_NAME'}, inplace=True)
agg_h_df = agg_h_df.loc[:, agg_cols]
# agg_h_df[agg_h_df['TM_KEY_DAY']==20240501]

# HH : D_CLUSTER
agg_hh_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_HH', 'D_CLUSTER']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_hh_df['AREA_NO'] = 4
agg_hh_df['AREA_TYPE'] = 'HH'
agg_hh_df['ORGID_HH'] = agg_hh_df['ORGID_HH'].astype(int).astype(str)
agg_hh_df.rename(columns={'ORGID_HH': 'AREA_CD'}, inplace=True)
agg_hh_df.rename(columns={'D_CLUSTER': 'AREA_NAME'}, inplace=True)
agg_hh_df = agg_hh_df.loc[:, agg_cols]
# agg_hh_df[agg_hh_df['TM_KEY_DAY']==20240601]

# Concat DataFrame
last_agg_all_df = pd.concat([agg_p_df, agg_g_df, agg_h_df, agg_hh_df], ignore_index=True)
last_agg_all_df['AGG_TYPE'] = 'S'
last_agg_all_df['FREQUENCY'] = 'Daily'
# last_agg_all_df['REMARK'] = 'Sales target DTAC Y2024(Jan-May) : as of 23-May-24(K.Voraphon)'
# last_agg_all_df['REMARK'] = last_agg_all_df['TM_KEY_MTH'].apply(lambda x: 'Sales target DTAC Y2024(Jan-May) : as of 23-May-24(K.Voraphon)' if x>=202401 and x<=202412 else 'Ref : K.Wanicha')
last_agg_all_df['REMARK'] = last_agg_all_df['TM_KEY_MTH'].apply(lambda x: 'Allocate from 64 Province (H Level)' if x>=202401 and x<=202403 else 'Allocate from 96 Cluster (HH Level)')

print(f'\nlast_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns')
# last_agg_all_df.loc[last_agg_all_df['TM_KEY_DAY']==20240601]
last_agg_all_df.tail(3)


last_agg_all_df : 424944 rows, 17 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE,AGG_TYPE,FREQUENCY,REMARK
424941,2024,202412,2025001,20241231,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,DTAC,T,4,HH,910096,SURIN,4.059910,125.857199,S,Daily,Allocate from 96 Cluster (HH Level)
424942,2024,202412,2025001,20241231,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,DTAC,T,4,HH,910097,UBON RATCHATHANI,20.606197,638.792107,S,Daily,Allocate from 96 Cluster (HH Level)
424943,2024,202412,2025001,20241231,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,DTAC,T,4,HH,910098,YASOTHON,0.139018,4.309551,S,Daily,Allocate from 96 Cluster (HH Level)


### Step 3 : Insert to "INTERIM_VINSIGHT_DATA"
    Delete -> Insert

In [29]:
''' Input Parameter '''

# Create list
month_list = last_agg_all_df['TM_KEY_MTH'].drop_duplicates().tolist()
mt_cd_list = last_agg_all_df['METRIC_CD'].drop_duplicates().tolist()

if len(mt_cd_list) == 1:
    mt_cd_list = str(mt_cd_list).replace(r'[', '(').replace(r']', ')')
else:
    mt_cd_list = tuple(mt_cd_list)

# Create Param
# v_param = dict(mth_start=202406, mth_end=202408, metric_cd=mt_cd_list)
v_param = dict(mth_start=min(month_list), mth_end=max(month_list), metric_cd=mt_cd_list)
v_target_schema = 'AUTOKPI'
v_target_table = 'INTERIM_VINSIGHT_DATA'

# query_delete = f"DELETE {v_target_schema}.{v_target_table} WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} AND METRIC_CD IN {v_param['metric_cd']}"
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} 
    AND METRIC_CD IN {v_param['metric_cd']}
"""

print(f"\nParameter...\n\n   -> TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}\n   -> METRIC_CD IN {v_param['metric_cd']}")
print(f'\nDataFrame...\n\n   -> last_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns') 
print(f'\nquery_delete...\n{query_delete}')


Parameter...

   -> TM_KEY_MTH BETWEEN 202401 AND 202412
   -> METRIC_CD IN ('DB1R000900CG', 'DB1R000900CS', 'DB1S000101CG', 'DB1S000101CS', 'DB2R010500CG', 'DB2R010500CS', 'DB2S010100CG', 'DB2S010100CS')

DataFrame...

   -> last_agg_all_df : 424944 rows, 17 columns

query_delete...

    DELETE AUTOKPI.INTERIM_VINSIGHT_DATA 
    WHERE TM_KEY_MTH BETWEEN 202401 AND 202412 
    AND METRIC_CD IN ('DB1R000900CG', 'DB1R000900CS', 'DB1S000101CG', 'DB1S000101CS', 'DB2R010500CG', 'DB2R010500CS', 'DB2S010100CG', 'DB2S010100CS')



In [30]:
''' Load Data '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Create rows from DataFrame
rows = [tuple(x) for x in last_agg_all_df.values]


# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
print(f'\nProcessing...')


try:
    # # Truncate
    # cur.execute(f"TRUNCATE TABLE {v_target_schema}.{v_target_table}")
    # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

    # Delete
    cur.execute(query_delete)
    print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
    # Insert
    cur.executemany(f"""
        INSERT INTO {v_target_table} 
        (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, DAY_VALUE, MTH_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)
        """, rows)
    print(f'\n   -> INSERT : "{v_target_table}" : Done !')

    cur.close()
    conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2025-03-24, 11:59:54

AKPIPRD : Connected

Processing...

   -> DELETE : "INTERIM_VINSIGHT_DATA" : Done !

   -> INSERT : "INTERIM_VINSIGHT_DATA" : Done !

AKPIPRD : Disconnected

Job Done !!!
